In [43]:
from pandas import read_csv, DataFrame
import json


def Керосин_сегодня(аэродром='UNWW', год_КТ=2021, год_Т=2024):
    """Функция возвращает среднюю расчётную стоимость ТС-1 на сегодня.
    Стоимость расчитывается как цена ТС-1 в аэропорте Толмачёво в крыло,
    умноженная на средний статистический коэффициент стоимости ТС-1
    в Толмачёво по годам к средней стоимости ТС-1 по России.
    Входная цента в рублях за тонну ТС-1."""
    # возврат = int(round(UNNT*коэффициент, 0))
    возврат = None
    try:
        with open('../json/Статистика.json', 'r', encoding='utf8') as f:
            d = json.load(f)
        коэффициент = d[аэродром]['КТ'].get(str(год_КТ), d[аэродром]['КТ']['средний'])
        цена = d["UNNT"]["ТС-1, руб/т без НДС"][str(год_Т)]
        возврат = int(round(цена/коэффициент, 0))
        del d
    except: print('[ОШИБКА] Проверить файл Статистика.json')
    finally: return возврат



def Себестоимость_лётного_часа(вылет="UNWW", прилёт="UNNT", тип="Ил-114-300"):
    возврат = {}
    крейсерская_скорость = None
    расход_на_лётный_час = None
    структура = {}
    самолёт = {}
    delta_t = None
    техническая_скорость = None
    продолжительность_полёта = None
    АвиаГСМ = None
    расстояние = None
    ценаАвиаГСМ = Керосин_сегодня(аэродром=вылет)
    #print('ценаАвиаГСМ = ', ценаАвиаГСМ)
    
    try:
        with open('../json/ИД.json', 'r', encoding='utf8') as f:
            d = json.load(f)
            структура = d["Структура себестоимости лётного часа"]
            del d
    except: print('[ОШИБКА] Проверить файл ИД.json')
    finally: ...
    
    try:
        with open('../json/Самолёты.json', 'r', encoding='utf8') as f:
            d = json.load(f)
        самолёт = d['Самолёты'][тип]
        крейсерская_скорость = самолёт["Крейсерская скорость, км/ч"]
        расход_на_лётный_час = самолёт["Расход ТС-1, кг/лётный час"]
        del d
    except: print('[ОШИБКА] Проверить файл Самолёты.json')
    finally: ...
    
    try:
        d = read_csv('../csv/FTD.csv', index_col='FT')
        расстояние = d.loc[вылет+прилёт]['D']
    except:
        print('[ОШИБКА] Проверить файл FTD.csv')
    finally:
        ...
    
    if тип in ["Ил-114-300", "ТВРС-44 Ладога"]:
        delta_t = 0.22
    elif тип in ["ЛМС-192 Освей", "ЛМС-901 Байкал"]:
        delta_t = 0.165
    
    техническая_скорость = (расстояние*крейсерская_скорость)/(расстояние + (крейсерская_скорость*delta_t))
    продолжительность_полёта = расстояние/техническая_скорость
    
    АвиаГСМ = продолжительность_полёта*расход_на_лётный_час
    процент = (АвиаГСМ*ценаАвиаГСМ)/(структура["АвиаГСМ"]*100000)
    возврат = {
        "АвиаГСМ": int(round(процент*структура["АвиаГСМ"]*100, 0)),
        "Амортизация": int(round(процент*структура["Амортизация"]*100, 0)),
        "Аэропортовое обслуживание": int(round(процент*структура["Аэропортовое обслуживание"]*100, 0)),
        "ТОиР СВАД": int(round(процент*структура["ТОиР СВАД"]*100, 0)),
        "Аэронавигация": int(round(процент*структура["Аэронавигация"]*100, 0)),
        "Агентские": int(round(процент*структура["Агентские"]*100, 0)),
        "Метео": int(round(процент*структура["Метео"]*100, 0)),
        "Аренда": int(round(процент*структура["Аренда"]*100, 0)),
        "Прочие": int(round(процент*структура["Прочие"]*100, 0)),
        "Страхование": int(round(процент*структура["Страхование"]*100, 0)),
        "Общие": int(round(процент*структура["Общие"]*100, 0)),
        "ФОТ": int(round(процент*структура["ФОТ"]*100, 0)),
        "Начисления": int(round(процент*структура["Начисления"]*100, 0))
    }
    возврат["Себестоимость"] = sum(возврат.values())
    return возврат

In [44]:
Себестоимость_лётного_часа()

#d = {}
#help(d)

{'АвиаГСМ': 45624,
 'Амортизация': 1825,
 'Аэропортовое обслуживание': 54749,
 'ТОиР СВАД': 23724,
 'Аэронавигация': 5475,
 'Агентские': 1825,
 'Метео': 1825,
 'Аренда': 5475,
 'Прочие': 9125,
 'Страхование': 1095,
 'Общие': 8030,
 'ФОТ': 18250,
 'Начисления': 5475,
 'Себестоимость': 182497}